In [1]:
import sys
sys.path.append("..") 
from utils.surface_gases import *

In [2]:
COUPLER_options = {}

COUPLER_options['Phi_global'] = 0.005
COUPLER_options['T_outgas'] = 1500.0

COUPLER_options['radius'] = 6.371e6 * 2.61
COUPLER_options['mass']   = 5.972e24 * 8.63
COUPLER_options['gravity'] = 6.67e-11 * COUPLER_options['mass'] / (COUPLER_options['radius']**2)
COUPLER_options['hydrogen_earth_oceans'] = 20.0
COUPLER_options['CH_ratio'] = 1.0
COUPLER_options['fO2_shift_IW'] = -4.0
COUPLER_options['nitrogen_ppmw'] = 2.0
COUPLER_options['planet_coresize'] = 0.55

solvevol_vols = ['H2O', 'CO2', 'N2', 'H2', 'CO', 'CH4']
for v in solvevol_vols:
    COUPLER_options[v+"_included"] = 0
    COUPLER_options[v+"_initial_bar"] = 0.0

COUPLER_options["H2O_included"] = 1
COUPLER_options["CO2_included"] = 1
COUPLER_options["N2_included"]  = 1
COUPLER_options["H2_included"]  = 1
COUPLER_options["CO_included"]  = 1
COUPLER_options["CH4_included"] = 1

# COUPLER_options["H2O_initial_bar"] = 1.0
# COUPLER_options["CO2_initial_bar"] = 0.0
# COUPLER_options["N2_initial_bar"]  = 0.0

print("Gravity = %.3f m.s-2" % COUPLER_options["gravity"])



Gravity = 12.433 m.s-2


In [3]:
COUPLER_options["mantle_mass"] = calc_mantle_mass(COUPLER_options)

In [4]:
solvevol_target = solvevol_get_target_from_params(COUPLER_options)
# solvevol_target = solvevol_get_target_from_pressures(COUPLER_options)

print("Targets:")
print(solvevol_target)

Targets:
{'H': 3.0999999999999995e+21, 'C': 3.0999999999999995e+21, 'N': 3.4050557736877863e+19}


In [5]:
p_d = solvevol_equilibrium_atmosphere(solvevol_target, COUPLER_options)

print(p_d)

p_bar = {}
for v in volatile_species:
    p_bar[v] = p_d[v+"_atm_bar"]

mubar = 0.0
ptot = 0.0
print("Partial pressures")
for v in solvevol_vols:
    print("    %s : %.3f bar"%(v, p_bar[v]))
    ptot += p_bar[v]
    mubar += p_bar[v]*molar_mass[v]
mubar /= ptot

print("mubar =",mubar)

m_d = {}
for v in solvevol_vols:
    m_d[v] = p_bar[v]*1.0e5 * 4 * 3.14 * (molar_mass[v]/mubar) * COUPLER_options["radius"]**2.0 / COUPLER_options["gravity"]

# print(solvevol_atmosphere_mass(p_bar, COUPLER_options))

print(" ")
print(m_d)

H_kg = (2*m_d["H2O"]/molar_mass["H2O"] + 2*m_d["H2"]/molar_mass["H2"] + 4*m_d["CH4"]/molar_mass["CH4"]) *molar_mass['H']
C_kg = (m_d["CO2"]/molar_mass["CO2"] + m_d["CO"]/molar_mass["CO"] + m_d["CH4"]/molar_mass["CH4"]) * molar_mass['C']
N_kg = m_d["N2"]

print(H_kg, C_kg, N_kg)


{'M_atm': 6.636804331673693e+21, 'P_surf': 237.4723930936772, 'H2O_atm_bar': 2.0085231577354685, 'H2O_atm_kg': 1.9907036948974784e+20, 'H2O_liquid_kg': 6.321700945703638e+19, 'H2O_solid_kg': 0.0, 'H2O_total_kg': 2.6228737894678423e+20, 'CO2_atm_bar': 0.010400476550053514, 'CO2_atm_kg': 2.518218269400503e+18, 'CO2_liquid_kg': 404531105134208.94, 'CO2_solid_kg': 0.0, 'CO2_total_kg': 2.518622800505637e+18, 'H2_atm_bar': 188.33355209350714, 'H2_atm_kg': 2.0887243820538415e+21, 'H2_liquid_kg': 0.0, 'H2_solid_kg': 0.0, 'H2_total_kg': 2.0887243820538415e+21, 'CH4_atm_bar': 44.259647358221294, 'CH4_atm_kg': 3.90571973319813e+21, 'CH4_liquid_kg': 5.21365144355274e+16, 'CH4_solid_kg': 0.0, 'CH4_total_kg': 3.9057718697125657e+21, 'CO_atm_bar': 2.642190357130186, 'CO_atm_kg': 4.071608147389149e+20, 'CO_liquid_kg': 3.5382319092794908e+16, 'CO_solid_kg': 0.0, 'CO_total_kg': 4.0719619705800766e+20, 'N2_atm_bar': 0.2180796505330692, 'N2_atm_kg': 3.361081392365819e+19, 'N2_liquid_kg': 4.39743814388839e

/Users/nichollsh/Projects/PROTEUS/tools/../utils/surface_gases.py:176: RuntimeWarning: invalid value encountered in log10
  ppmw = 10 ** (-0.738 + 0.876 * np.log10(p) - 5.44e-5 * p_total)
/Users/nichollsh/Projects/PROTEUS/tools/../utils/surface_gases.py:149: RuntimeWarning: overflow encountered in exp
  ppmw  = pb_N2**0.5 * np.exp(5908.0 * pb_tot**0.5/temp - 1.6*fO2_shift)
/Users/nichollsh/Projects/PROTEUS/tools/../utils/surface_gases.py:149: RuntimeWarning: invalid value encountered in scalar power
  ppmw  = pb_N2**0.5 * np.exp(5908.0 * pb_tot**0.5/temp - 1.6*fO2_shift)
/Users/nichollsh/Projects/PROTEUS/tools/../utils/surface_gases.py:76: RuntimeWarning: invalid value encountered in scalar power
  return const*p**exponent


In [6]:
H_mol = H_kg / (1.0e-3)
H_oc  = H_mol / (7.69e22 * 2.0)


print("[H] atm = %.4f" % H_oc)
print("C/H atm = %.4f" % (C_kg/H_kg))

[H] atm = 20.0998
C/H atm = 1.0023
